In [51]:
import pandas as pd 
import numpy as np
import math

In [26]:
def read_file(file):
    df = pd.read_csv(file, header=None)
    for column in df.columns:
        if int(column) == 0:
            df.drop(column, axis=1)
    shrunk_df = df.drop([0], axis=0)
    return shrunk_df

In [48]:
df = read_file("data/4clusters.csv")

### Create Dist Matrix

In [60]:
def euclideanDist(point, pointArray):
    return np.sqrt(np.sum((pointArray - point) ** 2, axis=1))

In [58]:
def calcDistMatrix(df, distFunctionVect):
    # must be fully numeric and normalized df
    dfarray = np.array(df)
    
    distMatrix = []
    for i, d in enumerate(dfarray):
        # performs Euclidean distance on all elements in data (vectorized)
        dists = distFunctionVect(dfarray[i], dfarray)
        distMatrix.append(dists)
    
    return pd.DataFrame(distMatrix)

In [67]:
dist_matrix = calcDistMatrix(df, euclideanDist)

### hclustering

In [ ]:
class Leaf:
    def __init__(self, type, height, data):
        self.type = type
        self.height = height
        self.data = data

    def __repr__(self):
        return f"type: {self.type}, height: {self.height}, data: {self.data}"

    def to_dict(self):
        json_dict = {}
        json_dict["type"] = self.type
        json_dict["height"] = self.height
        json_dict["data"] = self.data
        return json_dict

class Node:
        def __init__(self, type, height, nodes):
        self.type = type
        self.height = height
        self.nodes = nodes

    def __repr__(self):
        return f"type: {self.type}, height: {self.height}, nodes: {self.nodes}"

    def to_dict(self):
        json_dict = {}
        json_dict["type"] = self.type
        json_dict["height"] = self.height
        json_dict["nodes"] = self.nodes
        return json_dict


In [74]:
def hcluster(dist_matrix, threshold = np.inf):
    dist_matrix.replace(0, np.nan, inplace=True) #so that 0 isn't always reported as the min val

    min_locs = dist_matrix.idxmin()
    min_row = 0
    min_col = 0
    min_val = np.inf
    for val in min_locs:
        if dist_matrix.at[val, min_locs[val]] < min_val:
            min_col = val
            min_row = min_locs[val]
            min_val = dist_matrix.at[val, min_locs[val]]
    print(min_row, min_col, min_val)
            
        
    return

In [73]:
hcluster(dist_matrix)

18 20 1.4142135623730951
